In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gene-expression/sample_submission.csv
/kaggle/input/gene-expression/train (4).csv
/kaggle/input/gene-expression/train_labels.csv
/kaggle/input/gene-expression/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/gene-expression/train (4).csv")
test_df = pd.read_csv("/kaggle/input/gene-expression/test.csv")
train_labels_df = pd.read_csv("/kaggle/input/gene-expression/train_labels.csv")


In [4]:
train_df['Id'] = train_df['Id'].str.strip()
train_labels_df['Id'] = train_labels_df['Id'].str.strip()


In [5]:
merged_df = train_df.merge(train_labels_df, on='Id', how='inner')
#merge shape 
print(" Merge successful!")
print(f"Merged shape: {merged_df.shape}")
print(f"'Class' in columns? {'Class' in merged_df.columns}")


 Merge successful!
Merged shape: (150, 14575)
'Class' in columns? False


In [6]:
class_col = [col for col in merged_df.columns if 'Class' in col][0]
print(f" Found target column: {class_col}")


 Found target column: Class_x


In [7]:
#feature extraction
gene_cols = [col for col in merged_df.columns if col.startswith("gene_")]
X_train = merged_df[gene_cols]
y_train = merged_df[class_col]


In [8]:
X_test = test_df[gene_cols]
test_ids = test_df['Id']
print(f"Shapes -> X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}")

Shapes -> X_train: (150, 14572), y_train: (150,), X_test: (401, 14572)


In [9]:
merged_df = pd.merge(train_df, train_labels_df, on="Id", how="left")

In [10]:
gene_cols = [col for col in merged_df.columns if col.startswith("gene_")]

In [11]:
#correction

import pandas as pd

#load
train_df = pd.read_csv('/kaggle/input/gene-expression/train (4).csv')
train_labels = pd.read_csv('/kaggle/input/gene-expression/train_labels.csv')
test_df = pd.read_csv('/kaggle/input/gene-expression/test.csv')

#col names check
print(" Cols in train_df:", train_df.columns.tolist())
print(" Cols in train_labels:", train_labels.columns.tolist())

#check dup
if 'Class' in train_df.columns:
    train_df.drop(columns=['Class'], inplace=True)

#merge again
merged_df = pd.merge(train_df, train_labels, on='Id', how='inner')

# class
print("Merged", merged_df.columns.tolist())
print("Merge shape:", merged_df.shape)
print("'Class' in merged_df?", 'Class' in merged_df.columns)

 Cols in train_df: ['Id', 'gene_1', 'gene_3', 'gene_5', 'gene_7', 'gene_8', 'gene_9', 'gene_10', 'gene_11', 'gene_13', 'gene_14', 'gene_15', 'gene_16', 'gene_18', 'gene_19', 'gene_21', 'gene_23', 'gene_24', 'gene_25', 'gene_28', 'gene_30', 'gene_32', 'gene_33', 'gene_35', 'gene_41', 'gene_43', 'gene_46', 'gene_47', 'gene_48', 'gene_49', 'gene_50', 'gene_51', 'gene_52', 'gene_53', 'gene_54', 'gene_55', 'gene_57', 'gene_58', 'gene_60', 'gene_61', 'gene_62', 'gene_63', 'gene_64', 'gene_65', 'gene_67', 'gene_68', 'gene_69', 'gene_70', 'gene_71', 'gene_73', 'gene_74', 'gene_75', 'gene_76', 'gene_78', 'gene_79', 'gene_80', 'gene_81', 'gene_82', 'gene_83', 'gene_84', 'gene_85', 'gene_88', 'gene_89', 'gene_90', 'gene_91', 'gene_93', 'gene_94', 'gene_95', 'gene_96', 'gene_97', 'gene_98', 'gene_99', 'gene_100', 'gene_101', 'gene_102', 'gene_103', 'gene_104', 'gene_105', 'gene_107', 'gene_109', 'gene_111', 'gene_112', 'gene_113', 'gene_115', 'gene_117', 'gene_118', 'gene_120', 'gene_121', 'gene_1

In [12]:
gene_cols = [col for col in merged_df.columns if col.startswith("gene_")]
X_train_raw = merged_df[gene_cols].values
y_train = merged_df["Class"].values
X_test_raw = test_df[gene_cols].values
test_ids = test_df["Id"]


In [13]:
#knn imputation
from sklearn.impute import KNNImputer

print(" Imputing using KNN Imputer")
imputer = KNNImputer(n_neighbors=5)
X_train = imputer.fit_transform(X_train_raw)
X_test = imputer.transform(X_test_raw)
print(" Imputation done")



 Imputing using KNN Imputer
 Imputation done


In [14]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score



In [15]:
#base models rf,lgbm,lr
rf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42, n_jobs=-1)
lgbm = LGBMClassifier(n_estimators=300, max_depth=10, random_state=42)
lr = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial')



In [16]:
#voting classifier
voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('lgbm', lgbm), ('lr', lr)],
    voting='soft', 
    n_jobs=-1
)

#cross - val
print("5-Fold Stratified CV with Voting Classifier")
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]

    voting_clf.fit(X_tr, y_tr)
    y_pred = voting_clf.predict(X_val)
    f1 = f1_score(y_val, y_pred, average='macro')
    f1_scores.append(f1)
    print(f" Fold {fold+1} Macro F1: {f1:.4f}")

print(f"\n Final Avg Macro F1-Score with Voting Classifier: {np.mean(f1_scores):.4f}")


5-Fold Stratified CV with Voting Classifier
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070497 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 606839
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 14572
[LightGBM] [Info] Start training from score -1.742969
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -0.980829
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -1.696449
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

In [ ]:

submission = pd.DataFrame({
    'Id': test_ids,
    'Class': predictions
})

# Save submission file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file created!")
print(f"Submission shape: {submission.shape}")

In [18]:
voting_clf.fit(X_train, y_train)
predictions = voting_clf.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.079461 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 752243
[LightGBM] [Info] Number of data points in the train set: 150, number of used features: 14572
[LightGBM] [Info] Start training from score -1.752539
[LightGBM] [Info] Start training from score -1.752539
[LightGBM] [Info] Start training from score -0.985284
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -1.714798
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best